In [2]:
import sys
print(sys.executable)

c:\Users\dev\talk2doc\venv\Scripts\python.exe


#Loading BDE-small

In [3]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("BAAI/bge-small-en")

text = "Transformers use self-attention mechanisms."
embedding = model.encode(text, normalize_embeddings=True)

print(len(embedding))


c:\Users\dev\talk2doc\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading weights: 100%|██████████| 199/199 [00:00<00:00, 211.32it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: BAAI/bge-small-en
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


384


#CREATE FAISS INDEX

In [4]:
import faiss
import numpy as np

dimension = 384
index = faiss.IndexFlatIP(dimension)  # Inner Product (cosine since normalized)

# Example documents
documents = [
    "The capital of France is Paris.",
    "Transformers use self-attention.",
    "India's capital is New Delhi."
]

doc_embeddings = model.encode(
    ["passage: " + doc for doc in documents],
    normalize_embeddings=True
)

index.add(np.array(doc_embeddings))


Query Retrieval

In [5]:
query = "What is the capital of France?"

query_embedding = model.encode(
    "query: " + query,
    normalize_embeddings=True
)

D, I = index.search(np.array([query_embedding]), k=2)
for idx in I[0]:
    print(documents[idx])


The capital of France is Paris.
India's capital is New Delhi.


Inject into deepseek

In [9]:

import requests

context = "\n".join([documents[i] for i in I[0]])

print("Context retrieved:")
prompt = f"""
Use only the following context to answer.

Context:
{context}

Question:
{query}
"""
print("Sending request...")
response = requests.post(
    "http://localhost:11434/api/generate",
    json={
        "model": "deepseek-v3.1:671b-cloud",
        
        "prompt": prompt,
        "stream": False
    }
)
print("hello")

data = response.json()

if "response" in data:
    print(data["response"])
else:
    print("Error:", data)




Context retrieved:
Sending request...
hello
The capital of France is Paris.
